In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-25"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
15214,2024-10-25,República Tcheca 1. Liga,14:30,Sokol Prazsky,Polabi,1.49,2.38,155.5,1.84,1.82,-5.5,1.93,1.74,rVvuK7iJ,0.671141,0.420168,0.543478,0.549451,0.091309,276.100,129.380764,0.468601,0.6,1.341641,2.236068,242.88,3.904,1.214879,0.311188,-78.0,128.248,43.816169,0.341652,1.8,1.643168,0.912871,203.55,1.506,0.813837,0.540396,44.0,69,69,3.52,2.95,186.698,454.892,0.325233,0.007728,0.073215,-0.51,-0.102,-4.803922,0.643249,0.6,-0.043249,-2.61,-0.522,-2.643678,0.000000,0.0,0.000000
15215,2024-10-25,República Tcheca 1. Liga,15:00,Brno U23,Svitavy,5.39,1.10,166.5,1.84,1.82,12.5,1.99,1.70,SnGKWEB5,0.185529,0.909091,0.543478,0.549451,0.094620,230.282,192.241118,0.834807,1.8,1.643168,0.912871,99.68,3.176,2.931208,0.922924,2.0,152.672,65.139559,0.426663,2.4,1.341641,0.559017,101.20,1.712,0.698226,0.407842,38.0,89,92,1.12,1.10,134.198,156.006,0.934819,0.007728,0.111144,-0.72,-0.144,-30.486111,0.000000,0.0,0.000000,0.22,0.044,2.272727,0.692452,0.6,-0.092452
15216,2024-10-25,República Tcheca 1. Liga,15:00,Pardubice 2,Zlin,13.70,1.01,156.5,1.84,1.82,21.5,1.99,1.70,WKMBYzsg,0.072993,0.990099,0.543478,0.549451,0.063092,341.012,255.389296,0.748916,0.6,1.341641,2.236068,53.82,4.832,2.871188,0.594203,-109.0,263.150,122.039670,0.463765,2.4,1.341641,0.559017,436.59,2.994,1.596020,0.533073,69.0,39,81,1.38,5.39,317.292,333.322,1.220012,0.007728,0.111144,7.53,1.506,8.432935,0.000000,0.0,0.000000,-2.24,-0.448,-0.022321,0.278238,0.1,-0.178238
15217,2024-10-25,Suécia Basketligan,13:34,Sodertalje,Hogsbo,1.72,1.95,161.5,1.83,1.83,-2.5,1.99,1.70,ITE0yPo2,0.581395,0.512821,0.546448,0.546448,0.094216,230.884,45.744428,0.198127,1.8,1.643168,0.912871,211.64,3.036,0.521133,0.171651,5.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,74,87,2.86,1.82,195.322,0.000,0.088629,0.000000,0.111144,3.79,0.758,0.949868,0.457100,0.5,0.042900,0.00,0.000,inf,0.000000,0.0,0.000000
15218,2024-10-25,Romênia Liga Nacional Feminina,10:00,Rapid Bucuresti F,CSM Constanta F,1.69,2.00,146.5,1.81,1.85,29.5,1.99,1.69,bNxFj2DD,0.591716,0.500000,0.552486,0.540541,0.091716,291.236,136.146983,0.467480,1.2,1.643168,1.369306,263.76,4.560,3.077215,0.674828,-42.0,94.524,12.007745,0.127034,2.4,1.341641,0.559017,91.59,1.302,0.102567,0.078777,41.0,56,71,4.71,1.29,329.146,515.132,0.118809,0.015456,0.115289,-3.26,-0.652,-1.058282,0.000000,0.0,0.000000,21.46,4.292,0.232992,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15304,2024-10-25,China Cba,08:35,Guangzhou,Nanjing Tongxi,3.05,1.31,209.5,1.88,1.78,6.5,1.96,1.72,Q1vbe7te,0.327869,0.763359,0.531915,0.561798,0.091228,706.228,401.507784,0.568524,0.0,0.000000,NaN,787.50,7.910,4.267587,0.539518,-100.0,226.558,100.345709,0.442914,1.8,1.643168,0.912871,134.82,2.160,1.047187,0.484809,1.0,105,126,7.50,1.07,374.986,282.536,0.564388,0.038640,0.092231,-1.07,-0.214,-9.579439,0.563131,0.7,0.136869,-3.86,-0.772,-0.401554,0.432600,0.3,-0.132600
15305,2024-10-25,China Cba,08:35,Beijing Royal Fighters,Zhejiang Guangsha,1.60,2.13,207.5,1.83,1.83,-4.5,2.03,1.67,4njkcTBr,0.625000,0.469484,0.546448,0.5464

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
15257,14:45,Europa Euroliga,Fenerbahce,Panathinaikos,1.68,Back Home
15274,21:00,Eua Nba,Houston Rockets,Memphis Grizzlies,1.74,Back Home
